# Lets Data hit SIX

Working on different data sets to analyse and predict how cricket player will perform in a given match

### Step 1 : Data Loading and Understanding

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
batting_data=pd.read_csv('D:\\Cricket Data Analysis\\Data\\fact_bating_summary.csv')
bowling_data=pd.read_csv('D:\\Cricket Data Analysis\\Data\\fact_bowling_summary.csv')
match_data=pd.read_csv('D:\\Cricket Data Analysis\\Data\\dim_match_summary.csv')
player_info=pd.read_csv('D:\\Cricket Data Analysis\\Data\\dim_players.csv')


In [ ]:
# Batting Data
batting_data.head()

In [ ]:
batting_data.dtypes

In [ ]:
batting_data['SR']=batting_data['SR'].replace('-',0)

In [ ]:
def handle(value):
    return float(value.replace('',np.NaN).strip())

In [ ]:
batting_data['SR']=batting_data['SR'].astype(float)

In [ ]:
batting_data.shape

In [ ]:
batting_data.isna().sum()

In [ ]:
# Bowling Data

In [ ]:
bowling_data.head()

In [ ]:
bowling_data.dtypes

In [ ]:
bowling_data.shape

In [ ]:
# Match Data 


In [ ]:
match_data.head()

In [ ]:
match_data.dtypes

In [ ]:
match_data.shape

In [ ]:
match_data.isna().sum()

In [ ]:
match_data['margin']=match_data['margin'].fillna('Not_Available')

In [ ]:
# Player info

In [ ]:
player_info.head()

In [ ]:
player_info.drop(columns=['image'],axis=1,inplace=True)

In [ ]:
player_info.columns

In [ ]:
player_info.dtypes

In [ ]:
player_info.shape

## Step 2: Merge Data 

In [ ]:
merge_1=match_data.merge(batting_data,on='match_id')

In [ ]:
merged_data=merge_1.merge(bowling_data,on='match_id')

In [ ]:
merged_data.head()

In [ ]:
merged_data.shape

## Exploratory Data Analysis

In [ ]:
match_data=match_data.merge(batting_data,on='match_id')

In [ ]:
match_data.head()

In [ ]:
match_data.rename(columns={'team1':'TeamA','team2':'TeamB','winner':'Winner','margin':'Won_by','ground':'Location','matchDate':'Match_Date','match_id':'Match_id','match':'Match','teamInnings':'Playing','battingPos':'Batting_Position','runs':'Runs'},inplace=True)

In [ ]:
# Lets plot a histplot how many players scored how much runs 
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
sns.histplot(match_data['Runs'],bins=30)
plt.title('Distrubution how much runs each player scored per match')
plt.xlabel('Runs')
plt.ylabel('Frequency')

In [ ]:
# This shows that a lot of batsman failed to perform well only a few number of batsman score above 40 
# Top Scorers
top_scrores=match_data.sort_values(by='Runs',ascending=False)

In [ ]:
top_run_scores=top_scrores.head(10)

In [ ]:
# Most Consistent
consistent=match_data[(match_data['Batting_Position'] < 5) & (match_data['balls'] > 10)]
consistent=consistent.groupby('batsmanName')['SR'].mean().reset_index().sort_values(by='SR',ascending=False)

In [ ]:
consistent_players=consistent['batsmanName'].to_list()

In [ ]:
print(f'Top Batsman with consitent strike for T20 World Cup are ')
for i,player in enumerate(consistent_players[0:10]):
    print(f'{i+1}){player}')

In [ ]:
plt.bar(consistent_players[0:3],consistent['SR'][0:3])
plt.xlabel('BatsMan Name')
plt.ylabel('Strike Rate')
plt.title('Top 3 batsman of 2020 T20I World Cup')


In [ ]:
# Most wins by
winners=match_data.groupby('Match')['Winner'].value_counts().reset_index()

In [ ]:
winners.drop(columns='count',inplace=True)

In [ ]:
new_=winners['Winner'].value_counts().reset_index()

In [ ]:
new_.sort_values(by='count',ascending=False,inplace=True)

In [ ]:
plt.bar(list(new_['Winner'].unique()[0:4]),new_['count'][0:4])
plt.xlabel('Teams')
plt.ylabel('Number of Wins')
plt.title('Team with most wins')


In [ ]:
# England Pakistan SriLanka and India appears to be stronger team to win
location=match_data.groupby('Location')['Runs'].sum().reset_index()

In [ ]:
innings_played_stadium=[]
for loca in location['Location'].to_list() :
 innings_played_stadium.append(len(match_data[match_data['Location']==loca]['Match_id'].unique()))

In [ ]:
avg_score_at_pitches=[(x/y)//2 for x,y in zip(location['Runs'].to_list(),innings_played_stadium) ]

In [ ]:
avg_score_at_pitches

In [ ]:
avg_scores={} 
for x,y in zip(location['Location'].to_list(),avg_score_at_pitches):
  avg_scores[x]=y

In [ ]:
for key,value in avg_scores.items():
    print(f'Average Score of First Innings at {key} is {value}')

In [ ]:
plt.bar(location['Location'].to_list(),avg_score_at_pitches)
plt.title('Average Scores each innings at different stadium ')
plt.xlabel('Stadium')
plt.ylabel('Score')

In [ ]:
# Oppositon
teams=list(match_data['Playing'].unique())

In [ ]:
match_data_1=pd.read_csv('D:\\Cricket Data Analysis\\Data\\dim_match_summary.csv')

In [ ]:
# Now for bowlers
bowling_data=bowling_data.merge(match_data_1,on='match_id')

In [ ]:
sns.histplot(bowling_data['wickets'])
plt.title('Distribution of Wickets')
plt.xlabel('Wickets')
plt.ylabel('Frequency')

In [ ]:
# This graph indicates most of the bowlers failed to take wickets around 175+ meanwhile no bowler took 5 wickets 

In [ ]:
# most wickets by bowler
most_wickets=bowling_data.groupby('bowlerName')['wickets'].sum().reset_index()

In [ ]:
sorted_=most_wickets.sort_values(by='wickets',ascending=False)

In [ ]:
# Top 10 bowler with most wickets are 
print('Top 10 bowlers with most wicket')
for i,name in enumerate(sorted_['bowlerName'].to_list()[0:10]):
    print(f'{i+1}){name}')

In [ ]:
plt.bar(sorted_['bowlerName'][0:3],sorted_['wickets'][0:3])
plt.xlabel('Player Name')
plt.ylabel('Wickets Taken')
plt.title('Top 3 wicket taking bowlers in t20 world cup 2022')

In [ ]:
# Average bowling economy of each team 
bowling_economy=bowling_data.groupby('bowlingTeam')['economy'].mean().reset_index()

In [ ]:
# Most wicket taking team 
bowling_wickets=bowling_data.groupby('bowlingTeam')['wickets'].sum().reset_index()


In [ ]:
# Lets see average runs scored by each team 
average_run=match_data.groupby('Playing')['Runs'].sum().reset_index()

In [ ]:
# Matches Played by each team 
matches_played=match_data.groupby('Match_id')['TeamA'].value_counts().reset_index()

In [ ]:
nd_=matches_played['TeamA'].value_counts().reset_index()

In [ ]:
# Matches Played by each team 
matches_played_B=match_data.groupby('Match_id')['TeamB'].value_counts().reset_index()

In [ ]:
matches_played_B.rename(columns={'TeamB':'TeamA'},inplace=True)

In [ ]:
nd_1=matches_played_B['TeamA'].value_counts().reset_index()

In [ ]:
matches_play=nd_.merge(nd_1,on='TeamA',how='outer')

In [ ]:
matches_play.replace(np.NaN,float(0),inplace=True)

In [ ]:
matches_play['Match Played']=matches_play['count_x']+matches_play['count_y']

In [ ]:
matches_play.drop(columns=['count_x','count_y'],inplace=True)

In [ ]:
def handles(value):
    return int(value)

In [ ]:
matches_play['Match Played']=matches_play['Match Played'].apply(handles)

In [ ]:
matches_play.rename(columns={'TeamA':'Teams'},inplace=True)

In [ ]:
average_run.rename(columns={'Playing':'Teams'},inplace=True)

In [ ]:
strong=average_run.merge(matches_play,on='Teams')

In [ ]:
strong['Average Score Per Innings']=strong['Runs']//strong['Match Played']

In [ ]:
bowling_wickets.rename(columns={'bowlingTeam':'Teams'},inplace=True)

In [ ]:
team=strong.merge(bowling_wickets,on='Teams')

In [ ]:
bowling_economy.rename(columns={'bowlingTeam':'Teams'},inplace=True)

In [ ]:
team=team.merge(bowling_economy,on='Teams')

In [ ]:
team['Economy']=round(team['economy'],2)

In [ ]:
team.drop(columns=['economy'],inplace=True)

In [ ]:
new_.rename(columns={'Winner':'Teams'},inplace=True)

In [ ]:
team=team.merge(new_,on='Teams',how='outer')

In [ ]:
team.rename(columns={'count':'Wins'},inplace=True)

In [ ]:
team['Wins']=team['Wins'].fillna(float(0))

In [ ]:
team.dropna(inplace=True)

In [ ]:
team

In [ ]:
team=team.sort_values(by=['Average Score Per Innings','wickets'],ascending=False).reset_index()

In [ ]:
team.drop(columns=['index'],inplace=True)

In [ ]:
# Top 3 player with average innings per score
plt.bar(team['Teams'][0:3].to_list(),team['Average Score Per Innings'][0:3].to_list())
plt.xlabel('Team name')
plt.ylabel('Average Score Per Innings')
plt.title('Team with top average score per innings')

In [ ]:
plt.bar(team['Teams'][0:3].to_list(),team['Economy'][0:3].to_list())
plt.xlabel('Team name')
plt.ylabel('Economy')
plt.title('Team with Good Economy')

## Feature Engineering

In [ ]:
# Lets predict player would score in a match 
batting=pd.read_csv('D:\\Cricket Data Analysis\\Data\\fact_bating_summary.csv')

In [ ]:
batting=batting[batting['battingPos']<8]

In [ ]:
batting['SR']=batting['SR'].replace('-',0)

In [ ]:
batting['SR']=batting['SR'].astype(float)

In [ ]:
batting.select_dtypes(include=['int64','float64']).corr()

In [ ]:
batting.columns

In [ ]:
batting=batting.groupby('batsmanName')[['runs','4s','6s']].sum().reset_index()

In [ ]:
batting.head()

## Model Development

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(batting[['4s','6s']],batting['runs'],test_size=0.2,random_state=42)

In [ ]:
lr=LinearRegression()


In [ ]:
lr.fit(xtrain,ytrain)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
yhat=lr.predict(xtest)

### Model Evaluation

In [ ]:
sns.kdeplot(ytest,color='r')
sns.kdeplot(yhat,color='b')
plt.title('Comparision Between actual Values and Predicted values')

In [ ]:
lr.score(xtrain,ytrain)

In [ ]:
lr.score(xtest,ytest)

In [ ]:
from sklearn.model_selection import cross_val_score
rcross=cross_val_score(lr,batting[['4s','6s']],batting['runs'],cv=4)

In [ ]:
rcross

In [ ]:
alpha=[{'alpha':[0.1,1,10,1000]}]


## Ridge Regression Model

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
grid=GridSearchCV(Ridge(),param_grid=alpha)

In [ ]:
scaler=StandardScaler()

In [ ]:
grid.fit(scaler.fit_transform(xtrain),ytrain)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
grid.best_estimator_

In [ ]:
predict=grid.predict(scaler.fit_transform(xtest))

### Model Evaluation

In [ ]:
sns.kdeplot(ytest,color='r')
sns.kdeplot(predict,color='b')
plt.title('Comparision Between actual Values and Predicted values')

In [ ]:
grid.score(scaler.fit_transform(xtest),ytest)

## Compare Models

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# For regression Model 
print(f'MSE : {round(mean_squared_error(yhat,ytest),2)} , R2 value : {round(lr.score(xtest,ytest),2)}')

In [ ]:
# For Ridge Model
print(f'MSE : {round(mean_squared_error(ytest,predict),2)} , R2 value : {round(grid.score(scaler.fit_transform(xtest),ytest),2)}')

### Model Dumping

In [ ]:
import joblib
with open('model.pkl','wb') as f :
    joblib.dump(lr,f)

In [ ]:
with open('scaler.pkl','wb') as f:
    joblib.dump(scaler,f)